## Importation et modification données

In [ ]:
# Start writing code here...from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [ ]:
spotify_songs = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv')


Q1 = np.percentile(spotify_songs.duration_ms, 25,
                   interpolation = 'midpoint')
Q3 = np.percentile(spotify_songs.duration_ms, 75,
                   interpolation = 'midpoint')
IQR = Q3 - Q1

"""
mask = (spotify_songs.duration_ms < (Q3+1.5*IQR))
mask2 = spotify_songs.duration_ms > (Q1-1.5*IQR)
spotify_data_cleaned = spotify_songs.loc[mask]
spotify_data_cleaned = spotify_data_cleaned.loc[mask2]
"""
spotify_data_cleaned = spotify_songs
spotify_data_cleaned.track_album_release_date.str[:4]
spotify_data_cleaned.track_album_release_date.str[5:7]
spotify_data_cleaned = spotify_data_cleaned.assign(
    track_album_release_month = spotify_data_cleaned.track_album_release_date.str[5:7],
    track_album_release_year = spotify_data_cleaned.track_album_release_date.str[:4],
)
spotify_data_cleaned.drop(columns="track_album_release_date",inplace=True)
spotify_data_cleaned.track_album_release_year = spotify_data_cleaned.track_album_release_year.astype("int64")
"""
mask = (spotify_data_cleaned.track_popularity > 0)
spotify_data_cleaned = spotify_data_cleaned.loc[mask]
"""


'\nmask = (spotify_data_cleaned.track_popularity > 0)\nspotify_data_cleaned = spotify_data_cleaned.loc[mask]\n'

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def normalisation(data):
    columns = ["track_popularity","danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo", "duration_ms"]
    data_norme = data[columns]
    scaler = MinMaxScaler(feature_range=(0,1))
    data_norme = pd.DataFrame(scaler.fit_transform(data_norme), columns=columns)
    return data_norme


In [ ]:
def dedoublons(data):
    return data.drop_duplicates(subset ="track_name",keep = 'first', inplace=True)
    
dedoublons(spotify_data_cleaned)
spotify_data_cleaned

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,playlist_name,playlist_id,playlist_genre,playlist_subgenre,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_album_release_month,track_album_release_year
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,...,1,0.0583,0.102000,0.000000,0.0653,0.5180,122.036,194754,06,2019
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,...,1,0.0373,0.072400,0.004210,0.3570,0.6930,99.972,162600,12,2019
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,...,0,0.0742,0.079400,0.000023,0.1100,0.6130,124.008,176616,07,2019
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,...,1,0.1020,0.028700,0.000009,0.2040,0.2770,121.956,169093,07,2019
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,...,1,0.0359,0.080300,0.000000,0.0833,0.7250,123.976,189052,03,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32828,7bxnKAamR3snQ1VGLuVfC1,City Of Lights - Official Radio Edit,Lush & Simon,42,2azRoBBWEEEYhqV6sb7JrT,City Of Lights (Vocal Mix),♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,...,1,0.0936,0.076600,0.000000,0.0668,0.2100,128.170,204375,04,2014
32829,5Aevni09Em4575077nkWHz,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20,6kD6KLxj7s8eCE3ABvAyf5,Closer Remixed,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,...,1,0.0420,0.001710,0.004270,0.3750,0.4000,128.041,353120,03,2013
32830,7ImMqPP3Q1yfUHvsdn7wEo,Sweet Surrender - Radio Edit,Starkillers,14,0ltWNSY9JgxoIZO4VzuCa6,Sweet Surrender (Radio Edit),♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,...,0,0.0481,0.108000,0.000001,0.1500,0.4360,127.989,210112,04,2014
32831,2m69mhnfQ1Oq6lGtXuYhgX,Only For You - Maor Levi Remix,Mat Zo,15,1fGrOkHnHJcStl14zNx8Jy,Only For You (Remixes),♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,...,1,0.1090,0.007920,0.127000,0.3430,0.3080,128.008,367432,01,2014


## KNN

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

**Fonction pour appliquer la KNN**

In [ ]:
def KNN(data_normed_quanti,data):
    xx = data_normed_quanti
    yy = data.playlist_genre
    x_train, x_test, y_train, y_test = train_test_split(xx, yy, random_state=0)

    n_neighbors_list = np.unique(np.round(np.geomspace(1, 500, 100)).astype(int))
    param_grid = {"n_neighbors": n_neighbors_list}

    cls = KNeighborsClassifier()

    search = GridSearchCV(cls, param_grid, scoring="accuracy", cv=10)
    search.fit(xx, yy)
    kopt = search.best_params_
    print(kopt)
    
    knn = KNeighborsClassifier(n_neighbors = kopt["n_neighbors"])
    #training
    knn.fit(x_train,y_train)
    print(knn.score(x_test,y_test))

**Fonction pour appliquer KNN sur 2 genres**

In [ ]:
def KNN_2_genre(genre1,genre2):
    mask_1 = spotify_data_cleaned.playlist_genre == genre1
    mask_2 = spotify_data_cleaned.playlist_genre == genre2

    data_1= spotify_data_cleaned.loc[mask_1]
    data_2 = spotify_data_cleaned.loc[mask_2]
    
    data= pd.concat([data_1,data_2])

    data_normed_quanti = normalisation(data)
    
    KNN(data_normed_quanti,data)

**Fonction pour appliquer KNN sur 3 genres**

In [ ]:
def KNN_3_genre(genre1,genre2,genre3):
    mask_1 = spotify_data_cleaned.playlist_genre == genre1
    mask_2 = spotify_data_cleaned.playlist_genre == genre2
    mask_3 = spotify_data_cleaned.playlist_genre == genre3

    data_1 = spotify_data_cleaned.loc[mask_1]
    data_2 = spotify_data_cleaned.loc[mask_2]
    data_3 = spotify_data_cleaned.loc[mask_3]

    data= pd.concat([data_1,data_2,data_3])

    data_normed_quanti = normalisation(data)
    
    KNN(data_normed_quanti,data)

**Fonction pour appliquer KNN sur 4 genres**

In [ ]:
def KNN_4_genre(genre1,genre2,genre3,genre4):
    mask_1 = spotify_data_cleaned.playlist_genre == genre1
    mask_2 = spotify_data_cleaned.playlist_genre == genre2
    mask_3 = spotify_data_cleaned.playlist_genre == genre3
    mask_4 = spotify_data_cleaned.playlist_genre == genre4


    data_1 = spotify_data_cleaned.loc[mask_1]
    data_2 = spotify_data_cleaned.loc[mask_2]
    data_3 = spotify_data_cleaned.loc[mask_3]
    data_4 = spotify_data_cleaned.loc[mask_4]

    data= pd.concat([data_1,data_2,data_3,data_4])

    data_normed_quanti = normalisation(data)
    
    KNN(data_normed_quanti,data)

**Application sur tous les cas**

In [ ]:
print("EDM - RAP")
KNN_2_genre("edm","rap")
print("---------------------")

print("EDM - ROCK")
KNN_2_genre("edm","rock")
print("---------------------")

print("EDM - R&B")
KNN_2_genre("edm","r&b")
print("---------------------")

print("EDM - latin")
KNN_2_genre("edm","latin")
print("---------------------")

print("EDM - pop")
KNN_2_genre("edm","pop")
print("---------------------")

print("RAP - ROCK")
KNN_2_genre("rap","rock")
print("---------------------")

print("RAP - R&B")
KNN_2_genre("rap","r&b")
print("---------------------")

print("RAP - LATIN")
KNN_2_genre("rap","latin")
print("---------------------")

print("rap - pop")
KNN_2_genre("rap","pop")
print("---------------------")

print("rock - r&b")
KNN_2_genre("rock","r&b")
print("---------------------")

print("rock - latin")
KNN_2_genre("rock","latin")
print("---------------------")

print("rock - pop")
KNN_2_genre("rock","pop")
print("---------------------")

print("r&b - latin")
KNN_2_genre("r&b","latin")
print("---------------------")

print("r&b - pop")
KNN_2_genre("r&b","pop")
print("---------------------")

print("latin - pop")
KNN_2_genre("latin","pop")


EDM - RAP
{'n_neighbors': 9}
0.8656853725850966
---------------------
EDM - ROCK
{'n_neighbors': 15}
0.8658129175946548
---------------------
EDM - R&B
{'n_neighbors': 25}
0.867590027700831
---------------------
EDM - latin
{'n_neighbors': 41}
0.7971254836926479
---------------------
EDM - pop
{'n_neighbors': 19}
0.7812202097235462
---------------------
RAP - ROCK
{'n_neighbors': 14}
0.8942261038330908
---------------------
RAP - R&B
{'n_neighbors': 49}
0.7632850241545893
---------------------
RAP - LATIN
{'n_neighbors': 19}
0.7555448408871746
---------------------
rap - pop
{'n_neighbors': 17}
0.8129496402877698
---------------------
rock - r&b
{'n_neighbors': 32}
0.8185579196217494
---------------------
rock - latin
{'n_neighbors': 20}
0.8484669811320755
---------------------
rock - pop
{'n_neighbors': 25}
0.766750629722922
---------------------
r&b - latin
{'n_neighbors': 52}
0.6961876832844575
---------------------
r&b - pop
{'n_neighbors': 19}
0.7320622177621676
------------------

**Application sur les trois genres qui semblent le plus facile à prédire**

In [ ]:
KNN_3_genre("edm",'rap','rock')

{'n_neighbors': 9}
0.7931034482758621


**Tentative sur 4 genres**


In [ ]:
KNN_4_genre("edm",'rap','rock','latin')

{'n_neighbors': 34}
0.6638242894056847


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c2ad4461-7f35-42f5-b2d9-f686e8952f67' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>